In [16]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [17]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70


class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [18]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [19]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))

    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))

    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [22]:
env = gym.make("CartPole-v0")
# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-cartpole")

for iter_no, batch in enumerate(iterate_batches(
        env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = \
        filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

0: loss=0.704, reward_mean=21.4, rw_bound=28.5
1: loss=0.683, reward_mean=21.4, rw_bound=24.5
2: loss=0.675, reward_mean=17.1, rw_bound=18.0
3: loss=0.681, reward_mean=19.3, rw_bound=22.5
4: loss=0.659, reward_mean=21.3, rw_bound=22.0
5: loss=0.673, reward_mean=21.2, rw_bound=20.0
6: loss=0.660, reward_mean=30.2, rw_bound=33.5
7: loss=0.651, reward_mean=37.8, rw_bound=37.5
8: loss=0.627, reward_mean=52.1, rw_bound=58.0
9: loss=0.620, reward_mean=54.6, rw_bound=55.0
10: loss=0.620, reward_mean=59.9, rw_bound=70.5
11: loss=0.607, reward_mean=58.6, rw_bound=69.5
12: loss=0.618, reward_mean=46.3, rw_bound=59.0
13: loss=0.593, reward_mean=48.8, rw_bound=50.5
14: loss=0.606, reward_mean=43.0, rw_bound=49.5
15: loss=0.600, reward_mean=62.8, rw_bound=66.5
16: loss=0.587, reward_mean=59.2, rw_bound=70.5
17: loss=0.575, reward_mean=65.4, rw_bound=69.5
18: loss=0.578, reward_mean=62.4, rw_bound=72.5
19: loss=0.558, reward_mean=76.4, rw_bound=86.5
20: loss=0.573, reward_mean=72.3, rw_bound=87.5
21